[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Humboldt-WI/IPML/blob/master/tutorial_notebooks/t6_regression_solution.ipynb)


# A Supervised Learning Example: The linear Regression Model
<hr>

In [ ]:
# Standard imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## The California Housing data set

The "California Housing" data set is a widely used data set to demonstrate forecasting methods. As the name suggests, the data set concerns the valuation of real estate. It comprises socio-demographic information concerning the area of a property and a dependent (aka target) variable, which gives the median house value for California districts. This data was derived from the 1990 U.S. census. 

Being so popular, the data set is readily available in standard Python libraries. 

In [ ]:
# Downloading the data set
from sklearn.datasets import fetch_california_housing

california_housing = fetch_california_housing(as_frame=True)  # get the data as a Pandas dataframe

print(california_housing.DESCR)  # the data even comes with a nice description

The `sklearn` library provides the data in a specific format. Feature values and the target variable are already separated. The be consistent with our standard notation, we extract the data and store it, as usual, in variables $X$ and $y$. Of course, this is a good opportunity to also take a quick look into the data.

In [ ]:
X = california_housing.data
X.head(5)

In [ ]:
y = california_housing.target
y

### Descriptive Statistics

In [ ]:
# compute descriptive statistics that summarize the 
# target's distribution
y.describe()  

In [ ]:
X.describe() # do the same for all features

### Explanatory data analysis
To better understand the data, we consider the standard EDA operations introduced in the previous practical. They comprise the analysis of histograms and/or box-plots of the target and the features. 

Contrary to the previous demo in which our target variable was a binary class indicator, our target in the house value estimation setting is a real number. Therefore, we plots with grouping to, e.g., depict the distribution of a feature across different values of the target are inapplicable. Instead, we consider the (linear) correlation between features and the target to obtain some initial evidence as to which features might be important. 

While we only create the plots in the following, never forget that each plot and more generally every result deserves a careful analysis and discussion. Therefore, make sure to examine each plot and note down your key observations.

#### Target Variable: Medium House Value

In [ ]:
# Histogram of target 
plt.figure(figsize=(8, 6))
plt.hist(y, bins=30, edgecolor='k')
plt.xlabel(y.name)
plt.ylabel('Frequency')
plt.title(f'Distribution of {y.name}')
plt.show()

#### Features

The easiest way to produce a visualization of the distribution of all features is to call a `Matplotlib` plotting function via the data frame object. For example, to create a histogram of all (numerical) columns in the data frame, you can call:

In [ ]:
X.hist(figsize=(10,8))  # Increase figure size size the call will produce many sub-plots
plt.subplots_adjust(wspace=0.5,hspace=0.5)  # adjust the spacing of the sub plots
plt.show()

Being so convenient, it is tempting to use the same approach for other plots such as, for example, creating a boxplot of all features. However, this is less useful, as the below example shows. 

In [ ]:
# Boxplot of all features in a data frame
X.boxplot(figsize=(8,4)) 
plt.subplots_adjust(wspace=0.5,hspace=0.5)
plt.show()

So the problem lies in the different scale of the features. By default, `Pandas` uses a shared y-axis for all features. This renders the plot unreadable. We could remedy the problem that the feature values have different scale by first normalizing the feature values as discussed in the lecture on [Explanatory Data Analysis](https://moodle.hu-berlin.de/pluginfile.php/5910736/mod_resource/content/2/ipml_s3_eda.pdf). 

Alternatively, to create a boxplot for all (numerical) features, we can simply write a loop and plot each feature individually. This would nicely work when using `Matplotlib` but since we have to write a bit more code anyway, and obtain more control over our individual plots, we use `seaborn` in the below example.

In [ ]:
#3x3 matrix of box-plots for all the features
fig, axes = plt.subplots(3, 3, figsize=(15, 12))  # Create a 3x3 grid of subplots
axes = axes.flatten() # Flatten the axes for easier iteration

# Loop through each feature and create a box plot
for i, feature in enumerate(X.columns):
    sns.boxplot(x=X[feature], ax=axes[i], color='skyblue')
    axes[i].set_title(f'Box Plot of {feature}')
    axes[i].set_xlabel(feature)

# Remove empty subplots
for i in range(len(X.columns), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()  # Adjust layout
plt.show()

#### Correlation analysis
Having obtained an idea about the *univariate* distributions, we also take a look at the correlation structure in the data. For start, we consider the *Pearson* correlation coefficient, which we can compute for all pairs of features.

In [ ]:
# Correlation among the features
corr_matrix = X.corr()  
sns.heatmap(corr_matrix, cmap='vlag', annot=True)
plt.title('Feature correlation')
plt.show()

A powerful way to obtain more insight into the dependency structure of features is the `pairplot()`, which integrates a visualization of individual feature's distribution and their association. The latter is depicted using scatter plots.

In [ ]:
sns.pairplot(data=X)
plt.show()

Finally, it is also interesting - and in fact essential - to examine the correlation between features and the target variable. Since our target column is no longer part of the data frame, we can calculate (and visualize) this correlation as follows:

In [ ]:
# Correlation between individual features and the target
rho = X.corrwith(y)
# Visualization by means of a bar plot 
plt.barh(X.columns.tolist(), rho)
plt.title('Feature to target correlation')
plt.xlabel('Correlation')

## Linear Regression
Having completed our initial data screening and explorative analysis, we proceed by estimating a linear regression model to deepen our understanding of how the features and the target are related to another.

To that end, we consider the library `statsmodels`, a popular and powerful library for statistical modeling, which includes ordinary least squares (OLS) estimator. We provide all codes for the model estimation below.

In [ ]:
import statsmodels.api as sm 

# OLS model estimation
lr_main = sm.OLS(y,
               sm.add_constant(X)  # include an intercept
              ).fit() 

The library provides a neat function, `.summary()` to obtain a concise overview of the results of regression analysis. It includes key information like R-squared, estimated coefficients, standard errors, and p-values. This summary is crucial for evaluating model adequacy and feature significance.

In [ ]:
print(lr_main.summary())

# Exercises
Based on the above analysis, draw on your data science expertise to answer the following questions: 

## 1. What are the two most and the two least important features?
Briefly note what statistics/results you have considered to make your choice.

**Your answer:**


## 2. Scatter Plots
Create, for each of the selected features, a scatter plot of the selected feature against the target. Display these plots in a 2x2 grid

In [ ]:
# Code for plotting


## 3. Model Reestimation
Remove the two least important features from the data and reestimate the model. Briefly discuss whether this step has improved the model.

In [ ]:
# Code to reestimate the model using a new data set


**Did the model improve? Briefly discuss:**

## EXTRA: Backward Elimination 
Relying on the following pseudo code, implement a *backward elimination* procedure, in which you repetitively discard the least important feature from the model.

```
    Use all features as the current set of features  
    Do 
        Estimate a regression model using current set of features
        Store model performance in terms of a suitable statistic
        Identify the least important feature
        Discard that feature from current set of features
    Repeat untill all features but one have been deleted

```
Depict your results graphically by plotting the number of features in the regression model against model performance using the same statistic as in your backward elimination algorithm.
>Hint: given you have to run the code inside the above *Do ... Repeat* block multiple times, consider implementing this part as a custom function.

In [ ]:
# Backward elimination code:
# ---------------------------------------------------------------


## 5. Switching the Library
We have used `statsmodels` up to this point. However, the go-to library when it comes to machine learning is a different library called [scikit-learn](https://scikit-learn.org/stable/), typically abbreviated as sklearn.

Import that library. Then, using once more the data set with all features included, create another linear regression model using the class `linear_model.LinearRegression()`. Compare the coefficients between this model and the one we estimated above using `statsmodels.api.OLS`. They should be pretty much identical. Please verify that they are.

In [ ]:
# Code to estimate linear regression using sklearn.


## 6. Prediction 
We next use our regression model for prediction. Feeding it with data on feature values, the estimated regression coefficients facilitate forecasting real-estate prices. More specifically, the lecture has introduced you to the process of evaluating prediction performance and its key ingredients, indicators of forecast accuracy and practices to organize available data so as to mimic a real-world application of a model. 

<img alt="Holdout method" src="https://raw.githubusercontent.com/Humboldt-WI/demopy/main/holdout_method.png" width=600 height=800/>

As far as ingredient one is concerned, we will consider the mean squared error (MSE), defined as:<br> 
$ MSE = \frac{1}{n}\sum_{i=1}^{n} \left( Y_i - \hat{Y}_i \right)^2 $, <br>
with:
- $n$ = number of data points
- $Y$ = true values of the target variable
- $\hat{Y}$ = forecasts of the regression model

> Hint: if unsure how to implement the MSE yourself, you can be sure that ready-made functions are available to do it for you

As the equation shows, given a trained (regression) model, we need a way to compute predictions $\hat{Y}$. Your first task is to find a suitable function for this task. Use web-search, ask ChatGPT, or use some other way of your liking. Once you found a suitable function, write some code to try it out and understand how, in general, it works.

In [ ]:
# Your code to compute model predictions


Now that we know how, in principle, we can apply a trained model to (novel) data, we are ready to properly assess the model's accuracy. As shown by the above picture, this entails applying the model to some data, called the test in the above picture, then comparing model predictions to actual values of the target variable, and then aggregating the residuals using some error measure. We already agreed on using the MSE for the last step. Hence, unsurprisingly, your next task is implement the **holdout method**, which the picture shows. Specifically, write code to: 
1. Randomly split your data into a training set (70%) and a test set (30%)
2. Train a regression model using the training set
3. Compute forecasts for the test set
4. Compute the test set MSE of the regression model

> Hint: you have seen allmost everythihg that it takes to solve this task. The only missing bit is the partitioning of the data. You could write code to do it *manually* but given that this kind of data splitting is super common, there must be a ready-to-use function to do it and, as always, the real task is to find the right function to do the job.

In [ ]:
# 1. Randomly split your data into a training set (70%) and a test set (30%)


In [ ]:
# 2. Train a regression model using the training set


In [ ]:
# 3. Compute forecasts for the test set


In [ ]:
# 4. Compute the test set MSE of the regression model


In [ ]:
mse = np.sum(e)/len(e)
print(f'MSE of the regression models is {mse}')

Try to interpret the test set MSE of the model. What does the obtained value tell you?

Hopefully, thinking about this question has reminded you of the lecture, in which we discussed how the MSE is difficult to interpret. Therefore, compute the root mean-square error $\sqrt{MSE}$ and interpret this quantity. Do you agree that it is easier to interpret? 

In [ ]:
# Compute RMSE of the regression model
print('RMSE of the regression models is {}'.format(np.sqrt(mse)))

**Your interpretation of the RMSE:**


# 7. Cross-validation
The MSE of the regression model gives us a solid estimate of how well the model predicts medium house values, that is our target. Before diving into more advanced means of forecast accuracy evaluation, let's take a step back and check whether the assessment of the model on a holdout data set was necessary in the first place. To that end, write code to compute the training set RMSE of the model.

In [ ]:
# Estimate training set RMSE of the regression model
from sklearn.metrics import mean_squared_error

# Helper function
def rmse(y_true, yhat):
    """ Helpfer function to calculate root mean-square error """
    mse = mean_squared_error(y_true=y_true, y_pred=yhat)
    return np.sqrt(mse)

# Compute training set predictions
y_hat_tr = sk_lr.predict(Xtr)

rmse_tr = rmse(ytr, y_hat_tr)
print('Training set RMSE of the regression models is {}'.format(rmse_tr))

It is worth thinking about and discussing why the *training set* RMSE might not differ that much from the *test set* RMSE. This discussion, however, is out of the scope of this tutorial. Kindly accept that our result is a special case and that it would be very wrong to conclude that an evaluation of forecast accuracy on *fresh* data using the holdout method is dispensable. If you'd like to know more, we invite you to ask ChatGPT for its perspective on the matter. The very task of phrasing a suitable prompt would be an excellent exercise to help you to advance your ML skills. 

Provided we accept that we routinely assess forecast accuracy using out-of-sample data (that is any data but the training set), we should examine how the standard way to undertake this assessment. Entry k-fold cross-validation, which the lecture introduced as the process of randomly partitioning a data set into $k$ disjoint parts, training a model on the union of $k-1$ parts of the data, assessing the resulting model on the left out part, and repeating the process $k$ times while selecting a different part of the data as validation fold in each run: 
<br>

<img alt="Cross-validation" src="https://raw.githubusercontent.com/Humboldt-WI/demopy/main/cross_validation1.png" width=600 height=800/>

<br>
Once the process of training and assessing $k$ models has been completed, we can integrate the out-of-sample predictions and compute the cross-validation error of the model on the resulting data.
<br><br>

<img alt="Cross-validation-based performance assessment" src="https://raw.githubusercontent.com/Humboldt-WI/demopy/main/cross_validation2.png" width=600 height=800/>
<br>

The `sklearn` library supports k-fold cross-validation in several ways. Generally speaking, these options differ in how much flexibility they leave you to adjust the code and how easy they are to use. In this notebook, we begin the perhaps easiest approach toward cross-validation. To that end, we use the function `cross_val_score()` which is available in the module `sklearn.model_selection`.

In [ ]:
# Import the function cross_val_score from the sklearn module sklearn.module_selection
from sklearn.model_selection import cross_val_score


In [ ]:
# Study the documentation of the function cross_val_score to understand how to use it and compute the cross-validation performance of your regression model using that function.
?cross_val_score

In [ ]:
rmse_cv = cross_val_score(estimator=LinearRegression(), 
                X=X,  # Note how we must use the entire data set here, as the function does the partitioning internally and would otherwise cross-validate the training data
                y=y,
                cv=10,  # Ten-fold cross-validation
                scoring='neg_root_mean_squared_error'  # Specify the accuracy indicator explicitly 
               )
# The cross_val_score function assumes that we use accuracy measures; not error measures. Therefore, we have to multiply results by -1 to obtain estimates of the RMSE
rmse_cv = -1* rmse_cv
print('Cross-validation performance of the model in terms of RMSE:')
print(rmse_cv)
print('Average RMSE performance equals {:.2f}'.format(np.mean(rmse_cv)))